In [73]:
import pandas as pd
import tweepy
import json

In [74]:
with open("credentials.json") as f:
    credentials = json.load(f)

In [75]:
client = tweepy.Client(bearer_token=credentials['BEARER_TOKEN'])

In [76]:
tweets_df = pd.DataFrame()
users_df = pd.DataFrame()

In [77]:
# queries = ['#Polish', 'polish', '#Poland', 'Poland', '#NATO', 'NATO', 'Przewodów', '#WWIII']
queries = ['#Trump', 'Trump']

In [78]:
old_df = pd.read_csv('./data/Trump/tweets.csv')
old_df

,hashtag,id,author_id,text,like_count,reply_count,retweet_count,quote_count,created_at,lang,mentions
0,#Trump,1594626092526374912,1585188213496446977,Kenya's President Ruto &amp; His Dopey DP's Go...,0,0,0,0,2022-11-21 09:37:41+00:00,en,NaN
1,#Trump,1594625554375999488,19260815,#Trump - what a great Twitstraction.\nSlow cla...,1,1,0,0,2022-11-21 09:35:33+00:00,en,['44196397']
2,#Trump,1594625521744506880,925649047,@szampa56 @matteosalvinimi @elonmusk Ovvove! \...,0,0,0,0,2022-11-21 09:35:25+00:00,it,"['964460408', '270839361', '44196397']"
3,#Trump,1594625461685886982,1385036460735279113,"TRUMP LIE #25857 cont’d\n\n""...of the People. ...",0,1,0,0,2022-11-21 09:35:11+00:00,en,NaN
4,#Trump,1594625414886047746,1530045783231410178,Supreme Court Ignites Final Countdown in Texas...,0,0,0,0,2022-11-21 09:35:00+00:00,en,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2995,Trump,1594623902797672449,1375087130121269248,"@a_nnaschneider Das ist keine Meinung, sondern...",0,0,0,0,2022-11-21 09:28:59+00:00,de,['3292307982']
2996,Trump,1594623898443780097,1314621230281117738,#TrumpIsDone #ETTD @chiefehrman \n\n#𝗗𝗲𝗺𝗼𝗰𝗿𝗮𝘁 ...,1,0,0,0,2022-11-21 09:28:58+00:00,en,['387839868']
2997,Trump,1594623892814954497,1552085107455950848,@1ainTro11sBots @GeoffreyKeey @AndrewKnight226...,0,0,0,0,2022-11-21 09:28:57+00:00,en,"['17535349', '1269143398201475075', '171752108..."
2998,Trump,1594623890080284679,1578854753005703169,@joncoopertweets I disagree how can you compar...,0,0,0,0,2022-11-21 09:28:56+00:00,en,['27493883']


In [79]:
params = {
    'max_results': 100,
    'limit': 20,
    'tweet_fields': ['id', 'text', 'public_metrics', 'author_id', 'created_at', 'lang'],
    'user_fields': ['username', 'created_at', 'public_metrics', 'protected', 'verified'],
    'expansions': ['author_id', 'entities.mentions.username'],
#     'since_id': old_df.iloc[0]['id']
}

In [80]:
tweet_list = list()
users_list = list()

In [81]:
for query in queries:
    try:
        for chunk in tweepy.Paginator(client.search_recent_tweets, query=f'{query} -is:retweet', **params):
            for tweets in chunk.data:
                tweet = [query,
                         tweets.id,
                         tweets.author_id,
                         tweets.text,
                         tweets.public_metrics['like_count'],
                         tweets.public_metrics['reply_count'],
                         tweets.public_metrics['retweet_count'],
                         tweets.public_metrics['quote_count'],
                         tweets.created_at,
                         tweets.lang]

                mentions = None
                if tweets.entities is not None:
                    mentions = [mention['id'] for mention in tweets.entities['mentions']]
                tweet.append(mentions)

                tweet_list.append(tweet)

            for user in chunk.includes['users']:
                users_list.append([user.id,
                                   user.username,
                                   user.created_at,
                                   user.public_metrics['followers_count'],
                                   user.public_metrics['following_count'],
                                   user.public_metrics['tweet_count'],
                                   user.protected,
                                   user.verified])

        temp_tweets = pd.DataFrame(tweet_list,
                                   columns=['hashtag', 'id', 'author_id', 'text', 'like_count', 'reply_count',
                                            'retweet_count',
                                            'quote_count',
                                            'created_at', 'lang', 'mentions'])
        temp_users = pd.DataFrame(users_list,
                                  columns=['id', 'username', 'created_at', 'followers_count', 'following_count',
                                           'tweet_count', 'protected', 'verified'])

        tweets_df = pd.concat([tweets_df, temp_tweets])
        users_df = pd.concat([users_df, temp_users])
    except tweepy.TooManyRequests:
        print('WARNING: Tweet download stopped due to TooManyRequest exception. Wait 15 minutes...')
        break

In [82]:
tweets_df

,hashtag,id,author_id,text,like_count,reply_count,retweet_count,quote_count,created_at,lang,mentions
0,#Trump,1594632441796362240,82406130,I will be deleting my twitter account due to #...,0,0,0,0,2022-11-21 10:02:55+00:00,en,None
1,#Trump,1594632433072226305,795993746,#Musk und #Trump:\nwas für eine perfekte Freun...,0,0,0,0,2022-11-21 10:02:53+00:00,de,None
2,#Trump,1594632423207325697,1390752168584568838,That negative level of dignity... \n#Trump #Tr...,0,0,0,0,2022-11-21 10:02:51+00:00,en,None
3,#Trump,1594632355326607362,1275873611245662210,#Musk und (seine Anbiederung an) #Trump sind e...,0,0,0,0,2022-11-21 10:02:35+00:00,de,None
4,#Trump,1594632339900207104,1475140765579890697,Anche #Facebook riattiva #Trump. 🤔🤔🤔\n\nCosa s...,0,0,0,0,2022-11-21 10:02:31+00:00,it,None
...,...,...,...,...,...,...,...,...,...,...,...
3992,Trump,1594627328923303936,1417351230582521858,@hodgetwins Who are these girls that keep show...,0,0,0,0,2022-11-21 09:42:36+00:00,en,[18584875]
3993,Trump,1594627328898117632,1503343839515656192,@Allthetrueindi1 @chota_trump Follow me,1,1,0,0,2022-11-21 09:42:36+00:00,en,"[1271074744016629761, 1431952461833572358]"
3994,Trump,1594627327610306560,1579720363318988800,https://t.co/ytzkGXEEEr projection,0,0,0,0,2022-11-21 09:42:36+00:00,en,None
3995,Trump,1594627323269373952,1518917649480261633,@majornelson You folks moving to hive are just...,1,0,0,0,2022-11-21 09:42:35+00:00,en,[15913]


In [83]:
users_df

,id,username,created_at,followers_count,following_count,tweet_count,protected,verified
0,82406130,sorenmat,2009-10-14 17:07:23+00:00,146,340,5136,False,False
1,795993746,VictorVetterle,2012-09-01 12:35:44+00:00,472,511,74719,False,False
2,1390752168584568838,c345tEn56kOrmOa,2021-05-07 19:36:10+00:00,1037,1234,32633,False,False
3,1275873611245662210,ReckNelle,2020-06-24 19:29:27+00:00,859,1287,15595,False,False
4,1475140765579890697,Giancky26,2021-12-26 16:26:35+00:00,2287,2697,16215,False,False
...,...,...,...,...,...,...,...,...
5547,15913,majornelson,2006-11-22 20:28:03+00:00,1197289,10,67426,False,True
5548,378040682,bj2079,2011-09-22 14:58:47+00:00,1430,2865,12772,False,False
5549,17886107,PlanetAmerica,2008-12-05 01:28:53+00:00,405,405,1655,True,False
5550,1484377742049038341,jayson_moncada,2022-01-21 04:10:39+00:00,5,42,27,False,False


In [84]:
tweets_df.to_csv('./data/poland/tweets.csv', mode='a', index=False)
users_df.to_csv('./data/poland/users.csv', mode='a', index=False)